# スポーツジムの顧客データを分析３
ここでは既に退会してしまった顧客と継続して利用している顧客のデータを用いて、顧客の退会を予測します。  
決定木分析を用いて予測モデルを作成し、１ヶ月前の顧客データから翌月の退会予測を行います。


### このジムは、月末までに退会申請を提出することで、翌月末に退会することが出来ます。
- 退会前月(退会申請) &emsp;&nbsp;:&nbsp;2018年08月
- 退会月(退会申請済み) &nbsp;:&nbsp;2018年09月
- 退会 &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;:&nbsp;2018年10月

# ライブラリ

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection
from dateutil.relativedelta import relativedelta

## データを読み込み

In [ ]:
customer = pd.read_csv('customer_data.csv')       # 利用履歴を含んだ顧客データ
uselog_months = pd.read_csv('use_log_months.csv') # 利用履歴を年月、顧客毎に集計したデータ

## データ整形

In [ ]:
# 当月、１ヶ月前の利用回数の集計データ
year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

- 6ヶ月分のデータから予測した場合、ほんの数ヶ月辞めてしまう顧客を予測することができないので、当月と1ヶ月前のデータから予測します。

## 退会前月の退会顧客データを作成
退会の予測をする目的は、退会を未然に防ぐことです。  
退会申請を提出する1ヶ月前のデータから、翌月に退会申請を提出する確率を予測します。

In [ ]:
# 退会を未然に防ぐ為に退会申請を出す、前月のデータを作成し"uselog"と結合
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
print(len(uselog))
exit_uselog.head()

In [ ]:
# 欠損値除去
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

## 継続顧客のデータを作成

In [ ]:
# 継続顧客データとuselogデータを結合し、欠損値除去
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

- nameカラムが欠損しているデータは、退会前月データと結合できていないため除去します。

In [ ]:
# データをシャッフルし、"customer_id"が重複しているデータは最初のデータのみ取得
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

- 退会データに対して継続データが多すぎるので、重複しているデータを1つに絞ります。

In [ ]:
# 継続顧客データと退会顧客データを結合
predict_data = pd.concat([conti_uselog, exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

## 予測する月の在籍期間を作成

In [ ]:
# 在籍期間カラム作成
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

## 欠損値を除去

In [ ]:
predict_data.isna().sum()

In [ ]:
# "count_1"が欠損しているデータのみ除去
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

## 文字列型の変数を処理できるように整形
文字列データを機械学習に対応させるために、数値化します。

In [ ]:
# データ絞り込み
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

In [ ]:
# ダミー変数化
predict_data = pd.get_dummies(predict_data)
predict_data.head()

In [ ]:
# ダミー変数化時にできた、使わない変数を除去
del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]
del predict_data["gender_M"]
predict_data.head()

## 決定木を用いて退会予測モデルを作成

In [ ]:
#継続と退会のデータ量を揃える(比率 : 50対50)
exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

# データ結合し、モデル作成
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

In [ ]:
# y_test, y_test_predを格納
results_test = pd.DataFrame({"y_test":y_test ,"y_pred":y_test_pred })
results_test.head()

- 先頭の5行は全て正解しています。

## 予測モデルの評価を行ない、モデルをチューニング

In [ ]:
# results_testデータの正解率
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

- 約88%正解しています。

In [ ]:
# 学習用データと評価用データの精度
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

- 評価用データより、学習用データが約10%ほど高くなっているため過学習傾向にあると言えます。

In [ ]:
# 前回のモデルを修正
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

- 決定木の深さを5に設定し、モデルを簡易化することで精度の差を小さくしています。

## モデルに寄与している変数を確認

In [ ]:
# 説明変数毎に係数を表示
importance = pd.DataFrame({"feature_names":X.columns, "coefficient":model.feature_importances_})
importance

- １ヶ月前の利用回数が大きく寄与しており、その次に在籍期間、定期利用者かどうかが寄与していることが分かります。

## 顧客の退会を予測

In [ ]:
# 変数を定義
count_1 = 3
routing_flg = 1
period = 10
campaign_name = "入会費無料"
class_name = "オールタイム"
gender = "M"

In [ ]:
# ダミー変数を作成し、1つのリストに格納
if campaign_name == "入会費半額":
    campaign_name_list = [1, 0]
elif campaign_name == "入会費無料":
    campaign_name_list = [0, 1]
elif campaign_name == "通常":
    campaign_name_list = [0, 0]
if class_name == "オールタイム":
    class_name_list = [1, 0]
elif class_name == "デイタイム":
    class_name_list = [0, 1]
elif class_name == "ナイト":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routing_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [ ]:
# 作成したデータを元に予測を行う
print(model.predict([input_data]))
print(model.predict_proba([input_data]))

- 今回の場合、退会(1)が予測されました。  
- 2行目で、継続(0)と退会(1)の予測確率が出力されています。